# Nifty Behavior
When nifty opens too high or too low, does it maintain the direction throughout the day and for how long?
- How often nifty moves up and how often it moves down?
- In market shock, does market behave in a certain way?

In [2]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *
import pytz


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_START = dt.datetime.strptime("2015-02-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = INTERVAL_MIN1
EXCHANGE = EXCHANGE_NSE
pickle_file_name = f"{__name__}-2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_symbol_open(symbol, trade_dt, trade_tm, ohlc_type):
    data = ut.get_data(symbol=SYMBOL, date=trade_dt.date(), interval=INTERVAL, exchange=EXCHANGE)
    match = data.loc[data.index.time==trade_tm]
    if match.empty:
        return pd.NA
    if ohlc_type == "open":
        return data.loc[data.index.time==trade_tm].iloc[0].open
    elif ohlc_type == "close":
        return data.loc[data.index.time==trade_tm].iloc[0].close
    else:
        raise Exception(f"invalid ohlc_type {ohlc_type}")

def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

def get_symbol_first_candle_open(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open

def get_symbol_last_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[-1].close

market_open_tm = dt.time(hour=9, minute=16)
market_close_tm = dt.time(hour=15, minute=29)
train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["td_0916_open"] = train_dates.apply(lambda row: get_symbol_open(symbol=SYMBOL, trade_dt=row.name, trade_tm=market_open_tm, ohlc_type="open"), axis=1)
train_dates["td_0329_open"] = train_dates.apply(lambda row: get_symbol_open(symbol=SYMBOL, trade_dt=row.name, trade_tm=market_close_tm, ohlc_type="open"), axis=1)
train_dates["prev_0329_open"] = train_dates.td_0329_open.shift(1)

In [5]:
train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open
trade_date,,,,
2015-02-04,2015-02-03,8787.15,8707.35,None
2015-02-10,2015-02-09,8480.85,8571.05,8707.35
2015-02-16,2015-02-13,8831.35,8806.7,8571.05
2015-02-20,2015-02-19,8883.65,8830.25,8806.7
2015-02-24,2015-02-23,8748.2,8758.9,8830.25
...,...,...,...,...
2023-12-20,2023-12-19,21550.55,21095.15,21410.95
2023-12-21,2023-12-20,21023.65,21270.9,21095.15
2023-12-22,2023-12-21,21272.05,21333.4,21270.9


In [3]:
LOT_SIZE = 50
MARGIN_TIMES = 8
train_dates["long_profit"] = LOT_SIZE * (train_dates.td_0916_open - train_dates.prev_0329_open)
train_dates["short_profit"] = LOT_SIZE * (train_dates.td_0916_open - train_dates.td_0329_open)
train_dates["long_investment"] = (LOT_SIZE * train_dates.td_0916_open) / MARGIN_TIMES
train_dates["short_investment"] = (LOT_SIZE * train_dates.td_0916_open) / MARGIN_TIMES
train_dates["total_inv"] = train_dates["long_investment"] + train_dates["short_investment"]
train_dates["total_profit"] = train_dates["long_profit"] + train_dates["short_profit"]

In [4]:
"""
Test Data
Percentage profit on investment: 0.5598233376617603, total profit: 955259.9999999993, inv: 170635973.125
Percentage profit on long investment: 0.942389210522457, total profit: 804027.4999999997, inv: 85317986.5625
Percentage profit on short investment: 0.17243726197432716, total profit: 147119.99999999936, inv: 85317986.5625
Mean profit per trade: 875.5820348304302
Mean profit per long trade: 736.9637946837761
Mean profit per long trade: 134.23357664233518
"""
print(f"Percentage profit on investment: {train_dates.total_profit.sum() * 100 / train_dates.total_inv.sum()}, total profit: {train_dates.total_profit.sum()}, inv: {train_dates.total_inv.sum()}")
print(f"Percentage profit on long investment: {train_dates.long_profit.sum() * 100 / train_dates.long_investment.sum()}, total profit: {train_dates.long_profit.sum()}, inv: {train_dates.long_investment.sum()}")
print(f"Percentage profit on short investment: {train_dates.short_profit.sum() * 100 / train_dates.short_investment.sum()}, total profit: {train_dates.short_profit.sum()}, inv: {train_dates.short_investment.sum()}")
print(f"Mean profit per trade: {train_dates.total_profit.mean()}")
print(f"Mean profit per long trade: {train_dates.long_profit.mean()}")
print(f"Mean profit per long trade: {train_dates.short_profit.mean()}")

Percentage profit on investment: 0.4268011424061117, total profit: 1455079.999999999, inv: 340926922.5
Percentage profit on long investment: 0.6088504201365909, total profit: 1037867.4999999997, inv: 170463461.25
Percentage profit on short investment: 0.24617299034223386, total profit: 419635.00000000006, inv: 170463461.25
Mean profit per trade: 665.9405034324939
Mean profit per long trade: 474.5621856424324
Mean profit per long trade: 191.35202918376655


In [21]:
train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit,drawdown,long_drawdown,short_drawdown
trade_date,,,,,,,,,,,,,
2015-02-04,2015-02-03,8787.15,8707.35,None,NaN,3990.0,54919.69,54919.69,109839.38,NaN,NaN,NaN,3990.0
2015-02-10,2015-02-09,8480.85,8571.05,8707.35,-11325.0,-4510.0,53005.31,53005.31,106010.62,-15835.0,-15835.0,-11325.0,-4510.0
2015-02-16,2015-02-13,8831.35,8806.7,8571.05,13015.0,1232.5,55195.94,55195.94,110391.88,14247.5,14247.5,13015.0,1232.5
2015-02-20,2015-02-19,8883.65,8830.25,8806.7,3847.5,2670.0,55522.81,55522.81,111045.62,6517.5,6517.5,3847.5,2670.0
2015-02-24,2015-02-23,8748.2,8758.9,8830.25,-4102.5,-535.0,54676.25,54676.25,109352.5,-4637.5,-4637.5,-4102.5,-535.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-20,2023-12-19,21550.55,21095.15,21410.95,6980.0,22770.0,134690.94,134690.94,269381.88,29750.0,29750.0,6980.0,22770.0
2023-12-21,2023-12-20,21023.65,21270.9,21095.15,-3575.0,-12362.5,131397.81,131397.81,262795.62,-15937.5,-15937.5,-3575.0,-12362.5
2023-12-22,2023-12-21,21272.05,21333.4,21270.9,57.5,-3067.5,132950.31,132950.31,265900.62,-3010.0,-3010.0,57.5,-3067.5


In [5]:
print(train_dates.shape[0])
train_dates = train_dates.loc[train_dates.prev_0329_open.notna() & train_dates.prev_0329_open.notnull() & train_dates.td_0916_open.notna() & train_dates.td_0329_open.notna()].copy()
print(train_dates.shape[0])
train_dates["drawdown"] = train_dates.total_profit
train_dates["long_drawdown"] = train_dates.long_profit
train_dates["short_drawdown"] = train_dates.short_profit

for i, row in train_dates.iterrows():
    prev_row = train_dates.shift(1).loc[i]
    if not prev_row.empty and not pd.isnull(prev_row.td_0329_open):
        train_dates.at[i, "drawdown"] = prev_row.drawdown + row["total_profit"]
        train_dates.at[i, "long_drawdown"] = prev_row.long_drawdown + row["long_profit"]
        train_dates.at[i, "short_drawdown"] = prev_row.short_drawdown + row["short_profit"]


2201
2185


In [6]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit,drawdown,long_drawdown,short_drawdown
trade_date,,,,,,,,,,,,,
2015-02-03,2015-02-02,8828.35,8747.1,8799.65,1435.0,4062.5,55177.19,55177.19,110354.38,5497.5,5497.5,1435.0,4062.5
2015-02-04,2015-02-03,8787.15,8707.35,8747.1,2002.5,3990.0,54919.69,54919.69,109839.38,5992.5,11490.0,3437.5,8052.5
2015-02-05,2015-02-04,8708.15,8696.3,8707.35,40.0,592.5,54425.94,54425.94,108851.88,632.5,12122.5,3477.5,8645.0
2015-02-06,2015-02-05,8708.3,8668.1,8696.3,600.0,2010.0,54426.87,54426.87,108853.75,2610.0,14732.5,4077.5,10655.0
2015-02-09,2015-02-06,8579.4,8519.25,8668.1,-4435.0,3007.5,53621.25,53621.25,107242.5,-1427.5,13305.0,-357.5,13662.5
2015-02-10,2015-02-09,8480.85,8571.05,8519.25,-1920.0,-4510.0,53005.31,53005.31,106010.62,-6430.0,6875.0,-2277.5,9152.5
2015-02-11,2015-02-10,8602.7,8635.25,8571.05,1582.5,-1627.5,53766.88,53766.88,107533.75,-45.0,6830.0,-695.0,7525.0
2015-02-12,2015-02-11,8674.85,8725.4,8635.25,1980.0,-2527.5,54217.81,54217.81,108435.62,-547.5,6282.5,1285.0,4997.5
2015-02-13,2015-02-12,8736.55,8802.4,8725.4,557.5,-3292.5,54603.44,54603.44,109206.87,-2735.0,3547.5,1842.5,1705.0


In [7]:
"""
Test data
-52107.49999999911
-50729.99999999956
-10684.999999999536
"""
print(train_dates.drawdown.idxmin())
print(train_dates.drawdown.min())
print(train_dates.long_drawdown.min())
print(train_dates.long_drawdown.idxmin())
print(train_dates.short_drawdown.min())
print(train_dates.short_drawdown.idxmin())


2015-02-13 00:00:00
3547.5000000000364
-2277.5000000000546
2015-02-10 00:00:00
279.99999999992724
2015-02-19 00:00:00


In [8]:
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")


gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")


Month: 2015-02-28 00:00:00, profit: 19482.5, long profit: 11002.5, short_profit: 8480.0
Month: 2015-03-31 00:00:00, profit: 49947.5, long profit: 16447.5, short_profit: 33500.0
Month: 2015-04-30 00:00:00, profit: 13367.5, long profit: -405.0, short_profit: 13772.5
Month: 2015-05-31 00:00:00, profit: -6097.5, long profit: 2777.5, short_profit: -8875.0
Month: 2015-06-30 00:00:00, profit: -10820.0, long profit: -7025.0, short_profit: -3795.0
Month: 2015-07-31 00:00:00, profit: -6122.5, long profit: 3545.0, short_profit: -9667.5
Month: 2015-08-31 00:00:00, profit: 10800.0, long profit: -8485.0, short_profit: 19285.0
Month: 2015-09-30 00:00:00, profit: -2697.5, long profit: -2000.0, short_profit: -697.5
Month: 2015-10-31 00:00:00, profit: 23592.5, long profit: 14452.5, short_profit: 9140.0
Month: 2015-11-30 00:00:00, profit: -5277.5, long profit: -5027.5, short_profit: -250.0
Month: 2015-12-31 00:00:00, profit: 6915.0, long profit: 3530.0, short_profit: 3385.0
Month: 2016-01-31 00:00:00, pr